#### Немного об именнованных сущностях 

https://habr.com/ru/post/531940/

In [1]:
import ru_core_news_lg
import spacy as sp

from spacy.symbols import ORTH, LEMMA
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab
from spacy.matcher import Matcher

from spacy import displacy
from IPython.display import Image

from spacy.language import Language
from spacy_langdetect import LanguageDetector

nlp = sp.load('ru_core_news_lg')

In [13]:
# doc = nlp(u"Я хочу кислое зеленое яблоко.")

# displacy.render(doc, style='dep', jupyter=True)
doc = nlp("Я хочу уйти спать пораньше.")
displacy.render(doc, style='dep', jupyter=True)

# Для расшифровки названий тегов можно воспользоваться функций explain:
print("aux:", sp.explain("aux"))
print("PROPN:", sp.explain("PROPN"))
print("amod:", sp.explain("amod"))

aux: auxiliary
PROPN: proper noun
amod: adjectival modifier


In [15]:
doc = nlp(u'Скоро будет дождь в Ладожской. Артем, пойдем книжку читать!')
for ent in doc.ents:
    print(ent.text, ent.label_)

displacy.render(doc, style='ent')

Ладожской LOC
Артем PER


##### 4. Создание собственных шаблонов для поиска текста

In [4]:
Image(url="picture_2.png", width=500, height=300)

#### Попробуем создать собственный шаблон для распознавания последовательности токенов.

Matcher позволяет находить слова и фразы, используя правила, описывающие их атрибуты токенов. Правила могут ссылаться на аннотаций токенов (например, теги text или part-of-speech), а также на лексические атрибуты, такие как Token.

In [5]:
# объект vocab()— контейнер хранилища со словарными данными
matcher = Matcher(nlp.vocab)

# REGEX - регулярное выражение выбора

pattern = [
    {"IS_DIGIT": True}, 
    {"LOWER": "лондонский"},
    {"LOWER": {"REGEX": "(мировой|городской)"}},
    {"LOWER": "хакатон"},
]

matcher.add("Hakaton", [pattern])
doc = nlp(u'2022 Лондонский мировой хакатон был завершен победой команды - Лягушечки')

matches = matcher(doc)
print('matches:', matches)

for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span)

matches: [(16000734440979727948, 0, 4)]
2022 Лондонский мировой хакатон


#### Определение семантической близости
Два слова могут быть очень схожи по смыслу, но как измерить их близость? В подобных задачах на помощь могут прийти семантические вектора. Если два слова или многословных выражения похожи, то их вектора будут лежать близко друг к другу.

In [6]:
doc1 = nlp("Я люблю бургеры")
doc2 = nlp("Я люблю пиццу")

print(doc1.similarity(doc2))

doc3 = nlp("сон")
doc4 = nlp("любовь")

print(doc3.similarity(doc4))

0.7899125196385883
0.2275083311958443


### Создание своих собственных компонентов обработки

Модуль SpaCy поддерживает ряд встроенных компонентов (токенизатор, выделение именованных сущностей), но также позволяет определять свои собственные компоненты. По сути, компоненты – это последовательно вызывающиеся функции, которые принимают на вход документ, изменяют его и отдают обратно. Новые компоненты можно добавлять с помощью атрибута add_pipe:

In [7]:
''' 
@Language.factory('length_component')
def length_component(doc):
    doc_length = len(doc)
    print(f"Этот документ имеет длину токенов {doc_length}.")
    return doc

nlp.add_pipe('length_component', last = True)
print(nlp.pipe_names)
doc = nlp("Я не люблю вставать в понедельник в 6 утра.")
'''

' \n@Language.factory(\'length_component\')\ndef length_component(doc):\n    doc_length = len(doc)\n    print(f"Этот документ имеет длину токенов {doc_length}.")\n    return doc\n\nnlp.add_pipe(\'length_component\', last = True)\nprint(nlp.pipe_names)\ndoc = nlp("Я не люблю вставать в понедельник в 6 утра.")\n'

#### Обучение и обновление моделей



In [8]:
from spacy.lang.ru import Russian
import random

'''
nlp = sp.blank("ru")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
ner.add_label("Компания")
nlp.begin_training()



TRAINING_DATA = [
    ("Замечательный день сходить в Яблоко за кольцом", {"entities": [(29, 34, "COMPANY")]})
    ("Приготовила сегодня пирог с яблоком", {"entities": []})
    ("Потратила все деньги в Яблоке", {"entities": [(23, 28, "COMPANY")]})

]

nlp = Russian()

for i in range(10):
    random.shuffle(TRAINING_DATA)
    for batch in sp.util.minibatch(TRAINING_DATA):
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch]
        nlp.update(texts, annotations)
        
nlp.to_disk("model")

'''

'\nnlp = sp.blank("ru")\nner = nlp.create_pipe("ner")\nnlp.add_pipe(ner)\nner.add_label("Компания")\nnlp.begin_training()\n\n\n\nTRAINING_DATA = [\n    ("Замечательный день сходить в Яблоко за кольцом", {"entities": [(29, 34, "COMPANY")]})\n    ("Приготовила сегодня пирог с яблоком", {"entities": []})\n    ("Потратила все деньги в Яблоке", {"entities": [(23, 28, "COMPANY")]})\n\n]\n\nnlp = Russian()\n\nfor i in range(10):\n    random.shuffle(TRAINING_DATA)\n    for batch in sp.util.minibatch(TRAINING_DATA):\n        texts = [text for text, annotation in batch]\n        annotations = [annotation for text, annotation in batch]\n        nlp.update(texts, annotations)\n        \nnlp.to_disk("model")\n\n'

In [9]:
ner.add_label("Компания")
nlp.begin_training()

NameError: name 'ner' is not defined